<a href="https://colab.research.google.com/github/EddyDavies/Trade_With_Twitter/blob/main/ipynb/Transformer_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
try:
    from google.colab import drive
    IN_COLAB = True
    drive.mount('/content/drive')
except ModuleNotFoundError:
    IN_COLAB = False

Mounted at /content/drive


Check GPU

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Sep 11 23:27:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Check TPU

Check Memory

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!git clone https://github.com/EddyDavies/Trade_With_Twitter.git

Cloning into 'Trade_With_Twitter'...
remote: Enumerating objects: 332, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 332 (delta 192), reused 224 (delta 101), pack-reused 0
Receiving objects: 100% (332/332), 80.04 KiB | 10.00 MiB/s, done.
Resolving deltas: 100% (192/192), done.


In [4]:
!ls ../

bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr


In [5]:
%cd Trade_With_Twitter/src
!ls

/content/Trade_With_Twitter/src
bert_farm.ipynb  decorators.py	sentiment  utils.py
bert_farm.py	 prices		twitter


In [7]:
!pip install -r ../requirements.txt

     |████████████████████████████████| 2.8 MB 12.6 MB/s 
     |████████████████████████████████| 50 kB 7.6 MB/s 
     |████████████████████████████████| 895 kB 78.4 MB/s 
     |████████████████████████████████| 636 kB 77.8 MB/s 
     |████████████████████████████████| 3.3 MB 73.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
import os
import pandas as pd
from tqdm import tqdm

from utils import get_date_array, get_month_array
from sentiment.inference import to_dict_of_lists, get_paths

if IN_COLAB:
    source, results_folder = get_paths(data_folder='../../drive/MyDrive/ColabData')
else:
    source, results_folder = get_paths()

In [9]:
# 10% before "2017-09-19" and 50% after
# dates_range = ("2017-01-01", "2021-06-20")
dates_range = ("2017-09-19", "2021-06-20")
months_range = ["Jan 17", "Jun 21"]

months = get_month_array(months_range)
dates = get_date_array(dates_range)

In [13]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = model.to('cuda:0')

sentiment_analysis = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

In [11]:
from sentiment.inference import get_tweets, get_sentiments, save_sentiments, check_last_day

for date in dates:
    ids, tweets = get_tweets(date, source)

    percentage_per_chunk = 50
    save_every = 2000
    results = get_sentiments(date, tweets, ids, results_folder,
                              sentiment_analysis=sentiment_analysis,
                              save_every=save_every,
                              percentage_per_chunk=percentage_per_chunk)

    save_sentiments(ids, tweets, results_folder, date)
    check_last_day(results_folder, date)

2017-09-19:   0%|          | 0/12000 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
!ls ../

In [ ]:
# !rm -r ../data/bitcoin_scores

In [ ]:
print(results_folder)

In [ ]:
!git pull